In [213]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Read data






In [214]:
input_data=pd.read_csv("weather.csv")
input_data
#input_data.info()
#input_data.describe()

,location_id,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),sunrise (iso8601),sunset (iso8601)
0,0,6/1/2024,31.5,37.7,23.4,0.0,0.0,0.0,2024-06-01T01:19,2024-06-01T15:44
1,0,6/2/2024,27.8,34.4,20.3,0.0,0.0,0.0,2024-06-02T01:19,2024-06-02T15:45
2,0,6/3/2024,26.2,32.4,17.2,0.0,0.0,0.0,2024-06-03T01:19,2024-06-03T15:45
3,0,6/4/2024,26.7,31.9,18.8,0.0,0.0,0.0,2024-06-04T01:18,2024-06-04T15:46
4,0,6/5/2024,26.7,32.1,19.3,0.0,0.0,0.0,2024-06-05T01:18,2024-06-05T15:47
...,...,...,...,...,...,...,...,...,...,...
1093,2,5/28/2025,29.0,35.2,21.4,0.0,0.0,0.0,2025-05-28T01:25,2025-05-28T15:41
1094,2,5/29/2025,30.5,37.2,22.7,0.0,0.0,0.0,2025-05-29T01:25,2025-05-29T15:42
1095,2,5/30/2025,30.6,37.9,23.2,0.0,0.0,0.0,2025-05-30T01:25,2025-05-30T15:42
1096,2,5/31/2025,30.1,36.9,22.1,0.0,0.0,0.0,2025-05-31T01:24,2025-05-31T15:43


Pre Processing

In [215]:
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df


reduced_data=auto_data_type(input_data)

reduced_data["sunrise (iso8601)"] = pd.to_datetime(reduced_data["sunrise (iso8601)"]).dt.strftime("%H:%M")
reduced_data["sunset (iso8601)"] = pd.to_datetime(reduced_data["sunset (iso8601)"]).dt.strftime("%H:%M")


reduced_data

#reduced_data.isnull().sum()


,location_id,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),sunrise (iso8601),sunset (iso8601)
0,0,6/1/2024,31.500000,37.700001,23.400000,0.0,0.0,0.0,01:19,15:44
1,0,6/2/2024,27.799999,34.400002,20.299999,0.0,0.0,0.0,01:19,15:45
2,0,6/3/2024,26.200001,32.400002,17.200001,0.0,0.0,0.0,01:19,15:45
3,0,6/4/2024,26.700001,31.900000,18.799999,0.0,0.0,0.0,01:18,15:46
4,0,6/5/2024,26.700001,32.099998,19.299999,0.0,0.0,0.0,01:18,15:47
...,...,...,...,...,...,...,...,...,...,...
1093,2,5/28/2025,29.000000,35.200001,21.400000,0.0,0.0,0.0,01:25,15:41
1094,2,5/29/2025,30.500000,37.200001,22.700001,0.0,0.0,0.0,01:25,15:42
1095,2,5/30/2025,30.600000,37.900002,23.200001,0.0,0.0,0.0,01:25,15:42
1096,2,5/31/2025,30.100000,36.900002,22.100000,0.0,0.0,0.0,01:24,15:43
